In [2]:
import os,re,sys
sys.path.append('..')
import time,pickle
from tqdm import *
import numpy as np
import sklearn
import matplotlib.pylab as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2

from geocolab.Data_Utils import *

import gensim
from gensim import corpora, models, similarities
from pprint import pprint
from stop_words import get_stop_words  # load stopwords
import Stemmer  # Load an inplementation of the snow-ball stemmer

model_saved = os.path.join('../data')
abstractf = os.path.join(model_saved,'model_abstract','abstract')

import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.tools import FigureFactory as FF 
plotly.offline.init_notebook_mode()  
import seaborn as sns

from sklearn.manifold import TSNE
from sklearn.externals import joblib

ImportError: No module named geocolab.Data_Utils

American  Geophysical Union  (AGU) meeting  is a  geoscience conference
hold  each year  around Christmas  in San  Francisco. It  represents a
great opportunity for PhD students like  me to show off their work and
enjoy what the west coast has to offer. However, with  nearly 24 000 attendees,  AGU Fall Meeting is  also the
largest Earth  and space  science meeting  in the  world. As  such, it
represents an interesting data set to dive into the geoscience academic
world.  

In this post, I explore different information retrieval techniques taken from the field of natural language processing to explore the hidden patterns in the submitted abstract collection in 2015.

The objective is two fold:

- Identify semantic-based similarities between the contribution proposed at AGU to build a recommandation system based on the abstract content.
- Propose for each contributor a list of potential collaborators based on the authors of the papers proposed by our recommendantion system.

Different natural language processing tools are available in python to achieve this goal and after trying [sklearn](http://scikit-learn.org/stable/), I decided to settle on [gensim](https://radimrehurek.com/gensim/) which has partilarly fast and effective implementations to work with large dataset (~20000 abstracts here).

The basic stage, which I'll detail in the following are

- Cleaning the data.
- Construct a valid embedding for the corpus.
- Compute the similarities between the document within this embedding.

## Data cleaning

Data cleaning is an essential step for our recommendation system. Indeeed, our model is going to use the resulting corpus to build a consistent embedding of the abstracts and we don't want him to focus on unnecessary details. In particular, I used the module **unicodedata** to remove non-ascii caracters from the corpus.

In [ ]:
path_data = os.path.join(model_saved,'data_agu2015')
data = get_all_data(path_data)
sources = [df for df in data if (''.join(df.title) != "") and (df.abstract != '') and (len(df.abstract.split(' '))>100)]
sections = [df.section for df in sources]
abstracts = get_clean_abstracts(sources)
titles = get_clean_titles(sources)

In the following, I'll use on of my contributions to evaluate the consistency of our recommendation system. 

In [ ]:
def name_to_idx(sources,name):
    ''' From an authors, return the list of contributions '''
    contrib = [f for f in sources if name in f.authors.keys()]
    return [sources.index(elt) for elt in contrib]
    
my_contrib = name_to_idx(sources,'Clement Thorey')
print 'Title : %s'%(titles[my_contrib[0]])
print 'Abstract : %s'%(abstracts[my_contrib[0]])+'\n\n'

May be a bit of context can be usefull here. My PhD was about the detection and the characterization of magmatic intrusions on terrestrial planets. For those who wonder, a magmatic intrusion is a large volume of magma which, instead of rising until the surface and form a volcano, emplace at depth beneath the surface (less than a few km) where it cools and solidifies. On Earth, erosion and weathering can sometimes expose these intrusions at the surface. This is the case for instance in the henry mountains

![Example of an exposed magmatic intrusion in the Henry Mountains](https://upload.wikimedia.org/wikipedia/commons/a/a6/Laccolith_Montana.jpg)

My contributions at AGU deals with the detection and the characterization of those intrusions.

The first one is about the detection of a specific family of magmatic intrusions on the Moon which we call crater-centered intrusions. Particularly, those are magmatic intrusions that have emplaced and solidify beneath large impact craters (>20km in diameter) at the surface of the Moon. Consequently, these crater are heavily deformed due to the magmatic intrusion with large network of fracture crossing their floor.  In this contribution, I use machine learning techniques to try to automatically detect potential floor-fractured craters among 60000 referenced lunar impact craters. 


## Bag of Words model

The basic representation for a corpus of text document is called a [Bag of Word (BoW) model](https://en.wikipedia.org/wiki/Bag-of-words_model). This model looks at all the words in the corpus and first build a dictionary referencing all the words it has seen. Then, for each document in the corpus, in simply count how many times each word of the dictionary appears in this particular document. The result is a large matrix, each row is a text document, each columns is a particular word of the dicitonarry, that is, as you can guess, mostly fill with zeros. 

### Tokenizer

Under the hood, the BoW model assume an efficient tokenizer function which is able to split each document it its own set of tokens. A vanilla tokenizer function looks like this

In [ ]:
def tokenizer(text):
    return text.split(' ')

which simply look at each document and split it in a list of tokens according to the white spaces in the document. In the following, I'll use a slightly more evolve version of this tokenizer which I embedded in a Tokenizer class.

It use the [nltk](http://www.nltk.org/) library to first break each document (abstract) into sentences, then words.
Then using reg expression, it keeps only suitable tokens. In particular,

- `^[a-z]+$` keeps only words made of letters.
- `^[a-z][\d]$` selects tokens that have 2 characters, one letter, one number (molecule stuff).
- `^[a-z][\d][a-z]$` selects tokens that have 3 characters, one letter, one number, one letter (again molecule stuff).
- `^[a-z]{3}[a-z]*-[a-z]*$` includes some tokens that are composed of two words joined by -.

Next, I use a stopword list provide by **nltk** to filter out all the common word of the english language. Indeed, stopwords are words like 'the' or 'as' that are most likely present everywhere but do not carry meaningfull information in our purpose. This tokenizer also incorporates a last stage of stemming for each token. Stemming is the term used in information retrieval to describe the process for reducing words to their word stem, base or root form—generally a written word form. 

For instance, imagine this document

'Here we show that running is good for health. Indeed runner are quite healthy. Though they have runned a lot in their runly life, they are quite good at that.'

Clearly, this document is all about running! Nevertheless, without the stemming part in our tokenizer, 'runly' will have the same weight than 'good', equal to 1. In contrast, the stemming will reduce 'running', 'runned', 'runly' and 'runner' to their stem, namely 'run'. The word 'run' in the BoW will then have a weight of 4 for this document clearly underlying its importance ! I use the so-called SnowballStemmer included in the library **nltk** for stemming. 

Finally, in addition to these simple stem tokens, I also add the possibility to use bi-grams to the dictionary, i.e. all the combinations of two consecutive stem-words in each abstracts which is a common practise when using BoW model. We will see why later.


In [ ]:
class Tokenizer(object):
    ''' Class to handle the tokenization of a document.

    parameter:
    add_bigram (Boolean): Add the possibility to add bigram to the resulting
    list of tokens

    '''
    def __init__(self, add_bigram):
        self.add_bigram = add_bigram
        self.stopwords = get_stop_words('english')
        self.stopwords += [u's', u't', u'can', u'will', u'just', u'don', u'now']
        self.stemmer = Stemmer.Stemmer('english')

    def bigram(self, tokens):
        if len(tokens) > 1:
            for i in range(0, len(tokens) - 1):
                yield tokens[i] + '_' + tokens[i + 1]

    def tokenize_and_stem(self, text):
        tokens = list(gensim.utils.tokenize(text))
        filtered_tokens = []
        bad_tokens = []
        # filter out any tokens not containing letters (e.g., numeric tokens, raw
        # punctuation)
        for token in tokens:
            if re.search('(^[a-z]+$|^[a-z][\d]$|^[a-z]\d[a-z]$|^[a-z]{3}[a-z]*-[a-z]*$)', token):
                filtered_tokens.append(token)
            else:
                bad_tokens.append(token)
        filtered_tokens = [
            token for token in filtered_tokens if token not in self.stopwords]
        stems = map(self.stemmer.stemWord, filtered_tokens)
        if self.add_bigram:
            stems += [f for f in self.bigram(stems)]
        return map(str, stems)
    
tokenizer = Tokenizer(False)

### Dictionary

Then, the next step is to build the dictionary. **Gensim** is built in a memory-friendly fashion. Therefore, instead of loading the whole corpus into memory, tokenizing and stemming everything and see what remains, it allows us to build the dictionary document by document, with one document in memory at a time. 

In [ ]:
build = False
if build:
    # First, write the document corpus on a txt file, one document perline.
    write_clean_corpus(abstracts,abstractf+'_data.txt')
    tokeniser = Tokenizer(False)
    # Next create the dictionary by iterating of the abstract, one per line in the txt file
    dictionary = corpora.Dictionary(tokenizer.tokenize_and_stem(line) for line in open(abstractf+'_data.txt')) 
    dictionary.save(abstractf+'_raw.dict')
else:
    tokeniser = Tokenizer(False)
    dictionary = corpora.Dictionary.load(abstractf+'_raw.dict')

The resulting dictionary contains 70150 tokens. While we could work out a BoW model from there, it is often a good idea to remove extreme tokens. For instance, a token appearing in only 1 abstract is not going to help us build a recommandation system. Similarly, a token that appears in all the documents is not likely to carry meaningfull information neither for our purpose. I thereferore decided to remove all tokens that appear in less than 5 abstracts and in more than 80% of them. Note that creating the dictionarry can take up to 1 minute on my laptop which make serialization a good idea.

In [ ]:
build = False
if not os.path.isfile(abstractf+'_raw.dict') or build:
    dictionary =  corpora.Dictionary.load(abstractf+'_raw.dict')
    dictionary.filter_extremes(no_below=5,no_above=0.80,keep_n=200000)
    dictionary.id2token = {k:v for v,k in dictionary.token2id.iteritems()}
    dictionary.save(abstractf+'.dict')
else:
    dictionary = corpora.Dictionary.load(abstractf+'.dict')
    dictionary.id2token = {k:v for v,k in dictionary.token2id.iteritems()}

### BoW representation

Now we have the dictionary, it is actually easy to obtain the BoW representation of any document. We just have to tokenize the document using the same function used to build the dictionary and count the occurence of each word. Each dictionary in **gensim** possess a method **doc2bow** which does exactly that and return the representation as a sparse vector, i.e. a vector where only words that have a count different from zero are returned.

For instance, the BoW representation of my first abstract is 

In [ ]:
my_contrib_bow = dictionary.doc2bow(tokenizer.tokenize_and_stem(abstracts[my_contrib[0]]))
df = [f+(dictionary.id2token[f[0]],) for f in my_contrib_bow]
df = pd.DataFrame(df,columns = ['id','Count','Token']).sort_values(by='Count',ascending = False)
df.index= range(len(df))
table = FF.create_table(df.head(5))
#py.iplot(table, filename='Bow_0')
plotly.offline.iplot(table, show_link=False)

where the result are presented as a pandas dataframe for clarity and each id has been identified to its proper token. Indeed, each dictionary assign a unique integer id to all tokens appearing in the dictionary. Note that the BoW representation of my first abstract on Floor-Fractured craters, which underlies the importance of the stem token crater, lunar, intrusion, floor and classifi, is farely accurate.

By converting each abstract of the corpus in this doc2bow method, we can obtain the BoW representation of our full corpus. A careless memory way to do that is to just iterate the doc2bow method of our dictionary over the abstract list we have defined at the beginning. Nevertheless, this would end up storing the whole doc2bow representation into memory as a huge matrice. Instead, **gensim** has been designed such that it only requires that a corpus must be able to return one document vector (for instance, the doc2bow representation of the document here) at a time. We then define the BoW corpus as a sprecific object `MyCorpus` where the method `__iter__` is consistently defined to iter and transform each line of the txt file where the abstracts content is stored.

In [ ]:
class MyCorpus(Tokenizer):

    def __init__(self, name, add_bigram):
        super(MyCorpus, self).__init__(add_bigram)
        self.name = name
        self.load_dict()

    def load_dict(self):
        if not os.path.isfile(self.name + '.dict'):
            print 'You should build the dictionary first !'
        else:
            setattr(self, 'dictionary',
                    corpora.Dictionary.load(self.name + '.dict'))

    def __iter__(self):
        for line in open(self.name + '_data.txt'):
            # assume there's one document per line, tokens separated by
            # whitespace
            yield self.dictionary.doc2bow(self.tokenize_and_stem(line))

    def __str__(self, n):
        for i, line in enumerate(open(self.name + '_data.txt')):
            print line
            if i > n:
                break

In [ ]:
build = False
if build:
    bow_corpus = MyCorpus(abstractf,False)
    corpora.MmCorpus.serialize(abstractf+'_bow.mm',bow_corpus)
else:
    bow_corpus = corpora.MmCorpus(abstractf+'_bow.mm')

`MyCorpus` also posseses a print method which return the BoW representation of the first n document. Again, the return representation is parsed, i.e. it contains only the counts for non-zero element

In [ ]:
bow_corpus.__str__()

### Recommendation 

In the BoW representation of our corpus, each abstract is a point in a high-dimensional embedding (a 14669 dimensions embedding exactly). The *distance* or the *similarity* between one abstract and the rest of the corpus, according to some metrics, can then be used to compare different contributions together and then, to provide a recommendation list for a specific query. 

The euclidean distance is the more natural choice for the similarity measure. Given two vectors  $\vec{a}$ and $\vec{b}$, it is equal to 
$$d(\vec{a},\vec{b}) = \sqrt{(\vec{b}- \vec{a})\cdot(\vec{b}- \vec{a}) }$$


However, we'd like our distance to be independant of the magnitude of the difference between two vectors. For instance, we'd like to identify as similar two abstracts which contain exactly the same tokens even if their occurence differs significantly. The euclidean distance clearly does not have this property.

Accordingly, a more reliable measure for our purpose is called "the cosine similarity". For two vectors, $\vec{a}$ and $\vec{b}$, the cosine similarity $d$ is defined as :

$$ d(\vec{a},\vec{b})= \frac{\vec{a} \cdot \vec{b}}{|\vec{a}||\vec{b}|} = \cos(\vec{a},\vec{b})$$

In particular, this similarity measure is the dot product of the two normalized vector and hence, depends only on the angle between the two vectors (which is were its name comes from ;). It ranges from -1 when two vectors point in the opposite direction to 1 when they point in the same direction.

To compute the similarity of one query against our BoW representation, the natural procedure is to first transform our sparse representation into its dense equivalent, i.e. a matrice where the number of lines correspond to the number of tokens in the dictionary and the number of columns to the number of abstracts in the corpus. Then, we column normalize the matrice such that each document correspond to a unit vector in the representation space. Finaly, we take the dot product of the transposed matrice with the desired normalized query to get the cosine similarity agaist all documents in the corpus.

**Gensim** contains efficient utility functions to help converting from/to numpy matrice and therefore, this translates to

In [ ]:
def recom(abstractf,name):
    dictionary = corpora.Dictionary.load(abstractf+'.dict') 
    corpus = corpora.MmCorpus(abstractf + '_'+str(name)+'.mm')
    index = similarities.MatrixSimilarity.load(abstractf+'_'+str(name)+'.index')
    score = index[corpus[my_contrib[0]]]
    results = pd.DataFrame(np.stack((np.sort(score)[::-1],np.array(titles)[np.argsort(score)[::-1]])).T,
                       columns = ['CosineSimilarity','Title'])
    return results

df = recom(abstractf,'bow')
for i,row in df.iterrows():
    print 'Recom %d - Cosine: %1.3f - Title: %s'%(i+1,float(row.CosineSimilarity),row.Title)
    if i>8:
        break

## TF-IDF representation

One of the problem with the BoW representation is that it often puts too much weights on common words of the corpus. Indeed, while we remove most common words in english, i.e. the stopwords, word like 'present', 'show' of whatever is commonly use in the writing-abstract vocabulary can add some noise in regards to our purpose. In particular here, we would like to put more weights on tokens that make each abstract specific.

A common way to do this is to use a **Tf-Idf** normalization to re-weiht each count in the BoW representation by the frequency of the token in the whole corpus. **Tf** means term-frequency while **Tf–Idf** means term-frequency times inverse document-frequency. This way, the weight of common tokens in the corpus will be significantly lowered.

This implentation is available is **gensim** and can be easily combined with the BoW representation to get the representation of the corpus in the tf-idf space.


In [ ]:
build = False
if not os.path.isfile(abstractf+'_tfidf.mm') or build:
    # First load the corpus and the dicitonary
    bow_corpus = corpora.MmCorpus(abstractf+'.mm')
    dictionary = corpora.Dictionary.load(abstractf+'.dict')
    # Initialize the tf-idf model
    tfidf = models.TfidfModel(bow_corpus)
    # Compute the tfidf of the corpus itself
    tfidf_corpus = tfidf[bow_corpus]
    # Serialize both for reuse
    tfidf.save(abstractf+'_tfidf.model')
    corpora.MmCorpus.serialize(abstractf+'_tfidf.mm',tfidf_corpus)
else:
    tfidf = models.TfidfModel.load(abstractf+'_tfidf.model')
    tfidf_corpus = corpora.MmCorpus(abstractf+'_tfidf.mm')

In [ ]:
df = recom(abstractf,'tfidf')
for i,row in df.iterrows():
    print 'Recom %d - Cosine: %1.3f - Title: %s'%(i+1,float(row.CosineSimilarity),row.Title)
    if i>8:
        break

This indeed produces a slight improve of the score

## Latent Semantic Analysis (LSA) or (LSI)

And here comes  Latent Semantic Analysis (LSA) or  Indexing (LSI). LSI
is a common method in information retrieval to reduce the dimension of
the representation  space. The  idea behind  it is that  a lot  of the
dimensions  in  the  previous   representations  are  redundant.   For
instance,  the words  machine and  learning are  more likely  to occur
together. Therefore, shrinking these two  dimensions to only one which
is form  by a  linear combination  of the  token machine  and learning
would  reduce the  dimension without  any loss  of information.   More
generally, the Latent Semantic Analysis  aims to reduce the dimensions
while  keeping as  much  information possible  present  in the  higher
dimensal space by identifying deep semantic pattern in the corpus.

To identify this  semantic structure, Latent Semantic  Analysis used a
linear               algebra               method               called
[Singular Value Decomposition (SVD)](https://en.wikipedia.org/wiki/Latent_semantic_analysis). 
More formally, we know that the tf-idf representation can be written as a huge matric $X$ where each line corresponds to a token in the dictionary and each column corresponds to a document. Its size is (T,N) where T is the number of tokens and N the number of abstracts.

The maths behind LSI factorizes the matrice $X$ as

$$ X = UDV^T $$

where $U$ is a unitary matrix, i.e. formed by orthogonal unit norm vectors, of size (T,T), $D$ is a diagonal rectangular matrix of size (T,N) and $V$ is also a unitary matrix of size (N,N). The vectors in $U$ are called left eigenvectors, the vectors in $V$ are called right eigenvectors and the matrice $D$ is composed by their corresponding eigenvalues.

Particularly, this factorization has a nice property regarding the matrice containing all the cross-documents dot-product $X^TX$. Indeed,

$$X^TX = (UDV^T)^T(UDV^T) = (VD^TU^T)(UDV^T) = VD^TDV^T$$

and therefore, the orthonormal vectors in $V$ can be seen as the eigenvectors of the cross-document correlation matrice $X^TX$. As D is diagonal, $D^TD$ is just composed by the eigenvalues squared and therefore, these eigenvalues can be used as a direct proxy to evaluate the variance in the cross-document correlation matrice.

A rank $k$ approximation of $X$ can be obtain by

$$X_k = U_kD_kV^T_k$$

where $U_k$ and $V_k$ are the matrices $U$ and $V$ where we kept only the $k$ first eigenvectors, i.e size (T,K) and (N,K) respectively and $D_k$ is a squared matrice of size (K,K) which contains the k first eigenvalues of the diagonal. More importanly, $V_kD^T_k$ gives us a the new representation, called LSI space, where each document is characterized by k features. The   SVD  is  thus able  to   identify  a   consistent  lower-dimensional
approximation  of  the   higher-dimensional  tfidf  space. 

**Gensim**
implements  the   Latent  Semantic  Analysis  under   a  model  called
`LsiModel` which  can be  used on top  of our  previous representation
easily. It  requires a parameter, **num_topics**,  which corresponds to
the  desired   dimension  in  the   final  lsi  space.  I   settle  on
**num_topics=500** for good performance.

In [ ]:
build = False
if not os.path.isfile(abstractf+'_lsi.mm') or build:
    # First load the corpus and the dicitonary
    tfidf_corpus = corpora.MmCorpus(abstractf+'_tfidf.mm')
    dictionary = corpora.Dictionary.load(abstractf+'.dict')
    # Initialize the LSI model
    lsi = models.LsiModel(tfidf_corpus,id2word=dictionary, num_topics=500)
    # Compute the tfidf of the corpus itself
    lsi_corpus = lsi[tfidf_corpus]
    # Serialize both for reuse
    lsi.save(abstractf+'_lsi.model')
    corpora.MmCorpus.serialize(abstractf+'_lsi.mm',lsi_corpus)
else:
    lsi = models.LsiModel.load(abstractf+'_lsi.model')
    lsi_corpus = corpora.MmCorpus(abstractf+'_lsi.mm')

where we used $k=500$ here. The matrice $U$ and $D$ can be easily extracted from the model and we verify that the return lsi_corpus correspond to $V_kD_k^T$.

In [ ]:
V = gensim.matutils.corpus2dense(lsi_corpus, len(lsi.projection.s)).T / lsi.projection.s
lsi_corpus_dense = gensim.matutils.corpus2dense(lsi_corpus,len(lsi.projection.s))
np.allclose(np.dot(V,np.diag(lsi.projection.s)),lsi_corpus_dense.T)

In [ ]:
# Create traces
trace0 = go.Scatter(
    x = range(len(lsi.projection.s)),
    y = lsi.projection.s,
    mode = 'markers+lines',
    name = 'markers',
    marker = {'size':10}
)
layout = go.Layout(
    margin = {'b':30,'r':30,'l':30,'t':30},
    title='Eigenvalues of the SVD factorization',
    legend = {'yanchor':'auto',
              'bgcolor':'#EAEAF2',
              'xanchor':'auto',
             'font':{'size':9}})

data = [trace0]
fig = go.Figure(data=data,layout=layout)
# Plot and embed in ipython notebook!
plotly.offline.iplot(fig, show_link=False)
#py.iplot(fig, filename='scatter-mode')

In [ ]:
def print_n_topics(n,k):
    k -=1
    words = map(lambda x:dictionary.id2token[x],list(np.argsort(lsi.projection.u[:,k])[::-1])[:10])
    coeff = ['%1.3f'%(f) for f in list(np.sort(lsi.projection.u[:,k])[::-1])[:10]]
    return reduce(lambda x,y: y+' + '+x, [g+'x'+f for f,g in zip(words,coeff)][::-1])

In [ ]:
print_n_topics(10,42)

In [ ]:
lsi.show_topics(1)

In [ ]:
df = recom(abstractf,'lsi')
for i,row in df.iterrows():
    print 'Recom %d - Cosine: %1.3f - Title: %s'%(i+1,float(row.CosineSimilarity),row.Title)
    if i>8:
        break

## t-sne 

In [1]:
from palettable.tableau import Tableau_20
X_tsne = joblib.load(abstractf+'_Xtsne.pkl')
unique_section = set(sections)
color = Tableau_20.hex_colors
section2color = dict(zip(set(sections),color))
labels = np.array(sections)

ImportError: No module named palettable.tableau

In [ ]:
color = Tableau_20.hex_colors
from palettable.colorbrewer.sequential import BuPu_7
clever_color = dict.fromkeys(unique_section)
clever_color[u' SPA-Aeronomy']= sns.color_palette('Oranges').as_hex()[-2]
clever_color[u' SPA-Magnetospheric Physics']= sns.color_palette('Oranges').as_hex()[-2]
clever_color[u' SPA-Solar and Heliospheric Physics']= sns.color_palette('Oranges').as_hex()[-2]
clever_color[u' Planetary Sciences']= sns.color_palette('Oranges').as_hex()[-1]
clever_color[u' Geodesy']= sns.color_palette('Oranges').as_hex()[-3]
clever_color[u' Geomagnetism and Paleomagnetism']= sns.color_palette('Oranges').as_hex()[-4]
clever_color[u' Atmospheric Sciences']= sns.color_palette('Reds').as_hex()[-1]
clever_color[u' Atmospheric and Space Electricity']= sns.color_palette('Reds').as_hex()[-2]
clever_color[u' Cryosphere']= sns.color_palette('Blues').as_hex()[-1]
clever_color[u' Ocean Sciences']= sns.color_palette('Blues').as_hex()[-3]
clever_color[u' Hydrology']= sns.color_palette('Blues').as_hex()[-2]

clever_color[u' Biogeosciences']= sns.color_palette('Greens').as_hex()[-1]

clever_color[u' Earth and Planetary Surface Processes']= BuPu_7.hex_colors[-1]
clever_color[u' Natural Hazards']= BuPu_7.hex_colors[-2]
clever_color[u' Seismology']= BuPu_7.hex_colors[-3]
clever_color[u' Tectonophysics']= BuPu_7.hex_colors[-3]
clever_color[u' Volcanology, Geochemistry and Petrology']= BuPu_7.hex_colors[-4]

clever_color[u' Near Surface Geophysics']= Tableau_20.hex_colors[-3]
clever_color[u' Mineral and Rock Physics']= Tableau_20.hex_colors[-4]
clever_color[u" Study of Earth's Deep Interior"]= Tableau_20.hex_colors[-4]

clever_color[u' Nonlinear Geophysics']= Tableau_20.hex_colors[10]
clever_color[u' Earth and Space Science Informatics']= Tableau_20.hex_colors[11]

clever_color[u' Education']= Tableau_20.hex_colors[-5]
clever_color[u' Public Affairs']= Tableau_20.hex_colors[-5]
clever_color[u' Union']= Tableau_20.hex_colors[-5]
section2color = clever_color

In [ ]:
unique_section= [u' SPA-Aeronomy',u' SPA-Magnetospheric Physics',u' SPA-Solar and Heliospheric Physics',u' Planetary Sciences',
                u' Geodesy',u' Geomagnetism and Paleomagnetism',u' Atmospheric Sciences',u' Atmospheric and Space Electricity',
                u' Atmospheric and Space Electricity',u' Cryosphere',u' Ocean Sciences',u' Hydrology',u' Biogeosciences',
                u' Earth and Planetary Surface Processes',u' Natural Hazards',u' Seismology',u' Tectonophysics',
                u' Volcanology, Geochemistry and Petrology',u' Near Surface Geophysics',u' Mineral and Rock Physics',
                u" Study of Earth's Deep Interior",u' Nonlinear Geophysics',u' Earth and Space Science Informatics',
                u' Education',u' Public Affairs',u' Union']

In [ ]:
traces = []
for section in tqdm(unique_section,total = len(unique_section)):
    mask = labels == section
    x_data = X_tsne[mask,0]
    y_data = X_tsne[mask,1]
    text = np.array(titles)[mask] 

    trace = go.Scattergl(
        x = x_data,
        y = y_data,
        text = [f[:50]+'...' for f in text],
        name = section,
        hoverinfo = 'name+text',
        mode = 'markers',
        marker = dict(
            size = 5,
            opacity = 0.5,
            color = section2color[section],
            line = {'width':0}
        )
        )
    traces.append(trace)

layout = go.Layout(
    xaxis = {'showline':False,
             'showticklabels':False,
             'zerolinewidth':0,
             'showgrid':False,
            'domain':[0,0.7]},
    yaxis = {'showline':False,
            'showticklabels':False,
            'zerolinewidth':0,
            'showgrid':False},
    height = 600,
    width = 900,
    margin = {'b':30,'r':30,'l':30,'t':30},
    title='t-sne of the 500 LSA embedding',
    legend = {'yanchor':'auto',
              'bgcolor':'#EAEAF2',
              'xanchor':'auto',
             'font':{'size':9}})

data = traces
fig = go.Figure(data=data,layout=layout)
#plotly.offline.iplot(fig, show_link=False)
py.iplot(fig,filename = 'Tsne_LSA_AGU')

## Final wrapper 

In [ ]:
def recom(query):
    ''' Final recommendation system 
    
    Input:
    - abstractf : path where to load the dictionary,lsi corpus, and index corpus.
    
    ''' 
    # Load the necessary models, the lsi corpus and the correpsonding index
    dictionary = corpora.Dictionary.load(abstractf+'.dict') 
    tfidf = models.TfidfModel.load(abstractf+'_tfidf.model')
    lsi = models.LsiModel.load(abstractf+'_lsi.model')
    corpus = corpora.MmCorpus(abstractf + '_lsi.mm')
    index = similarities.MatrixSimilarity.load(abstractf+'_lsi.index')
    
    #Transform the query in lsi space
    ## Transform in the bow representation space
    vec_bow = dictionary.doc2bow(tokeniser.tokenize_and_stem(query))
    ## Transform in the tfidf representation space
    vec_tfidf = tfidf[vec_bow]
    ## Transform in the lsi representation space
    vec_lsi = lsi[vec_tfidf]
    ## Get the cosine siminalrity of the query agaisnt all the abstracts
    cosine = index[vec_lsi]
    ## Sort them and return a nice dataframe
    results = pd.DataFrame(np.stack((np.sort(cosine)[::-1],np.array(titles)[np.argsort(cosine)[::-1]])).T,
                       columns = ['CosineSimilarity','Title'])
    return results

## Collaborators search 

In [ ]:
path_data = os.path.join(model_saved,'data_agu2015')
annuary = get_all_contrib(path_data)
index_authors = pd.DataFrame([[f.name,f.country,f.link] for f in annuary],columns = ['name','country','link'])
idx_authors = index_authors.groupby('name')

In [ ]:
# Dictionary which map each title to another dictionary
# Containing the different absract contributors together with som
# useful information

def get_country(name):
    try: 
        country = idx_authors.get_group(name).country.values[0].upper()
    except:
        country = ''
    return country
authors = {titles[i]:{key:
                      {'inst':inst,
                       'link':obj.link,
                       'title':titles[i],
                       'country':get_country(key)}
                      for key, inst in obj.authors.iteritems()} for i,obj in enumerate(sources)}
def collab_based_on_n_abstract(query,n = 5):
    ''' Return a list of the potential contributors based
    on the n first abstract proposed by the recommendation 
    system '''
    df = recom(query)
    df = df.head(n)
    collab = {}
    for i,row in df.iterrows():
        collab.update(authors[row.Title])
    for name,description in collab.iteritems():
        print '%s from the %s, %s'%(name.upper(),description['inst'],idx_authors.get_group(name).country.values[0].upper())
        print 'Based on his/her abstract untitled'
        print ' %s'%(description['title'])
        print '%s \n'%(description['link'])

In [ ]:
collab_based_on_n_abstract(abstracts[my_contrib[0]],1)